In [2]:
# Load my Packages
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
# Assign hidden API key
with open("api_stuff.txt", "r") as file:
    api_key = file.read()

In [13]:
# Extract Season info from API
urls = "https://api-nba-v1.p.rapidapi.com/seasons"

headers = {
	"X-RapidAPI-Key": api_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

seasons = requests.get(urls, headers=headers)
s = seasons.json()['response']

In [15]:
# Extract Teams info from API
urlt = "https://api-nba-v1.p.rapidapi.com/teams"

headers = {
	"X-RapidAPI-Key": api_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

teams = requests.get(urlt, headers=headers)
t_df = pd.DataFrame(teams.json()['response'])

In [24]:
# Get players info from API
urlp = "https://api-nba-v1.p.rapidapi.com/players"

querystring = {"team":"1", "season":"2021"}

headers = {
	"X-RapidAPI-Key": api_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

players = requests.get(urlp, headers=headers, params=querystring)
p_df = pd.DataFrame(players.json()['response'])

In [31]:
# Get Player Stats per game info from API
urlps = "https://api-nba-v1.p.rapidapi.com/players/statistics"

querystring = {"id":"236","season":"2020"}

headers = {
	"X-RapidAPI-Key": api_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

player_stats = requests.get(urlps, headers=headers, params=querystring)
ps = pd.DataFrame(player_stats.json()['response'])

In [34]:
# Create my desired table